In [15]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [16]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [17]:
feedback_bits = 256
ncodebooks=1024
ncentroids=16
train_sam_num = 3000 # 训练集样本数

In [18]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



In [19]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Mithral
learn_multisplits(): initial loss:    0.4114035790956755
learn_multisplits(): returning loss:  0.034108580905012786
learn_multisplits(): initial loss:    0.22865395002152872
learn_multisplits(): returning loss:  0.02819026226643473
learn_multisplits(): initial loss:    0.16674691382049406
learn_multisplits(): returning loss:  0.024378076195716858
learn_multisplits(): initial loss:    0.5722808308993843
learn_multisplits(): returning loss:  0.06736493739299476
learn_multisplits(): initial loss:    3.832544771251533
learn_multisplits(): returning loss:  0.5254806191421497
learn_multisplits(): initial loss:    5.55149107510136
learn_multisplits(): returning loss:  0.858807616867125
learn_multisplits(): initial loss:    3.5477621711205685
learn_multisplits(): returning loss:  0.5494255996309221
learn_multisplits(): initial loss:    2.532438226167656
learn_multisplits(): returning loss:  0.46435750368982553
learn_multisplits(): initial loss:    0.6969303916331653
le

/data/hdr/pq/bolt/experiments/python/clusterize.py:1436: UserWarning: Persisting input arguments took 0.83s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  _learn_mithral_initialization(X, ncodebooks, pq_perm_algo='start')


X_res mse / X mse:  0.16049072
fitting dense lstsq to X_res


: 

: 

In [ ]:
x_test = np.load(dir_train+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [ ]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_fb%i_cb%i_ct%i.npy' % (feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.1054532  -0.13105224 -0.30966505 ...  0.13274463 -0.10357921
   0.16380036]
 [-0.29332012 -0.3815415  -0.43490967 ... -0.43085617  0.14691003
  -0.02406657]
 [ 0.39552528 -0.06842993 -0.18442045 ... -0.18036693  0.33477697
   0.28904498]
 ...
 [ 0.20765835  0.05681469 -0.81064355 ... -0.05512231 -0.04095689
   0.10117806]
 [ 0.14503604  0.11943699 -0.12179814 ... -0.05512231  0.14691003
   0.16380036]
 [-0.29332012  0.1820593  -0.30966505 ... -0.30561155 -0.29144612
   0.10117806]]
(32000, 256)
